In [11]:
import time
import sys
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split, KFold
import multiprocessing
import os

In [2]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
 
    return False

In [38]:
def process_weather():
    weather_file = pd.read_csv("Kaggle-Walmart-Sales-Prediction-in-Stormy-Weather-master/Data/weather.csv", sep=',')
    weather_data = weather_file.drop('codesum', axis = 1) # codesum 컬럼 삭제
    
    data = weather_data.values[:,2:] # index, station_nbr, date를 제외한 나머지 데이터 array로 변경
    # 데이터가 있을 경우 float형으로 변경, 데이터가 없을 경우 unicodedata로 변경
    for col in range(data.shape[1]):
        for row in range(data.shape[0]):
            if not is_number(data[row, col]):
                if row == 0:
                    i = 1
                    while not is_number(data[row+i, col]):
                        i += 1
                    data[row, col] = data[row+i, col]
                else:
                    data[row, col] = data[row-1, col]
            data[row, col] = float(data[row, col]) # Convert string to float value

    weather_processed = np.append(weather_file.values[:,0:2], data, axis = 1)
    df = pd.DataFrame(weather_processed, index=None, columns=weather_data.columns.values)
    df.to_csv("Kaggle-Walmart-Sales-Prediction-in-Stormy-Weather-master/Data/weather_processed.csv", sep=',', index=None)

In [30]:
weather_file

station_nbr        date tmax tmin tavg depart dewpoint wetbulb heat  \
0                1  2012-01-01   52   31   42      M       36      40   23   
1                2  2012-01-01   48   33   41     16       37      39   24   
2                3  2012-01-01   55   34   45      9       24      36   20   
3                4  2012-01-01   63   47   55      4       28      43   10   
4                6  2012-01-01   63   34   49      0       31      43   16   
5                7  2012-01-01   50   33   42      M       26      35   23   
6                8  2012-01-01   66   45    M      M       34      46    M   
7                9  2012-01-01   34   19   27      M       17      23   38   
8               10  2012-01-01   73   53   63      M       55      58    2   
9               11  2012-01-01   72   48   60      7       54      56    5   
10              12  2012-01-01   72   48   60      M       54      57    5   
11              13  2012-01-01   38   18   28      M       12      21   37   
12              14  2012-01-01   50   34   42      5       25      35   23   
13              15  2012-01-01   48   26   37     16       35      38   28   
14              16  2012-01-01   50   30   40      M       37      40   25   
15              17  2012-01-01   56   38   47      M       23      37   18   
16              18  2012-01-01   59   40   50      4       28      40   15   
17              19  2012-01-01   38   25   32     10       26      30   33   
18              20  2012-01-01   59   41   50      M       28      40   15   
19               1  2012-01-02   50   31   41      M       26      35   24   
20               2  2012-01-02   46   28   37     12       24      32   28   
21               3  2012-01-02   45   24   35     -1       11      28   30   
22               4  2012-01-02   60   37   49     -2       26      39   16   
23               6  2012-01-02   60   30   45     -4       27      38   20   
24               7  2012-01-02   44   26   35      M       17      29   30   
25               8  2012-01-02   59   36    M      M       29      40    M   
26               9  2012-01-02   21   11   16      M        8      15   49   
27              10  2012-01-02   65   45   55      M       33      46   10   
28              11  2012-01-02   63   40   52     -1       31      44   13   
29              12  2012-01-02   65   47   56      M       33      46    9   
...            ...         ...  ...  ...  ...    ...      ...     ...  ...   
20487           11  2014-10-30   81   59   70      4       63      65    0   
20488           12  2014-10-30   81   61   71      M       63      66    0   
20489           13  2014-10-30   61   32   47      M       26      38   18   
20490           14  2014-10-30   77   45   61      5       39      50    4   
20491           15  2014-10-30   56   30   43      0       36      39   22   
20492           16  2014-10-30   57   34   46      M       39      43   19   
20493           17  2014-10-30   77   43   60      M       41      50    5   
20494           18  2014-10-30   79   56   68      5       51      58    0   
20495           19  2014-10-30   51   32   42     -4       33      40   23   
20496           20  2014-10-30   80   52   66      M       50      57    0   
20497            1  2014-10-31   53   40   47      M       36      41   18   
20498            2  2014-10-31   49   40   45      0       35      40   20   
20499            3  2014-10-31   55   33   44    -12       32      41   21   
20500            4  2014-10-31   76   56   66     -1       47      56    0   
20501            5  2014-10-31    M    M    M      M       37       M    M   
20502            6  2014-10-31   76   51   64      0       47      55    1   
20503            7  2014-10-31   58   38   48      M       32      41   17   
20504            8  2014-10-31   78   55   67      M        M       M    0   
20505            9  2014-10-31   39   25   32      M       18      27   33   
20506           10  2014-10-3

In [29]:
weather_processed = pd.read_csv("Kaggle-Walmart-Sales-Prediction-in-Stormy-Weather-master/Data/weather_processed.csv", sep=',')
weather_processed

station_nbr        date  tmax  tmin  tavg  depart  dewpoint  wetbulb  \
0                1  2012-01-01  52.0  31.0  42.0    16.0      36.0     40.0   
1                2  2012-01-01  48.0  33.0  41.0    16.0      37.0     39.0   
2                3  2012-01-01  55.0  34.0  45.0     9.0      24.0     36.0   
3                4  2012-01-01  63.0  47.0  55.0     4.0      28.0     43.0   
4                6  2012-01-01  63.0  34.0  49.0     0.0      31.0     43.0   
5                7  2012-01-01  50.0  33.0  42.0     0.0      26.0     35.0   
6                8  2012-01-01  66.0  45.0  42.0     0.0      34.0     46.0   
7                9  2012-01-01  34.0  19.0  27.0     0.0      17.0     23.0   
8               10  2012-01-01  73.0  53.0  63.0     0.0      55.0     58.0   
9               11  2012-01-01  72.0  48.0  60.0     7.0      54.0     56.0   
10              12  2012-01-01  72.0  48.0  60.0     7.0      54.0     57.0   
11              13  2012-01-01  38.0  18.0  28.0     7.0      12.0     21.0   
12              14  2012-01-01  50.0  34.0  42.0     5.0      25.0     35.0   
13              15  2012-01-01  48.0  26.0  37.0    16.0      35.0     38.0   
14              16  2012-01-01  50.0  30.0  40.0    16.0      37.0     40.0   
15              17  2012-01-01  56.0  38.0  47.0    16.0      23.0     37.0   
16              18  2012-01-01  59.0  40.0  50.0     4.0      28.0     40.0   
17              19  2012-01-01  38.0  25.0  32.0    10.0      26.0     30.0   
18              20  2012-01-01  59.0  41.0  50.0    10.0      28.0     40.0   
19               1  2012-01-02  50.0  31.0  41.0    10.0      26.0     35.0   
20               2  2012-01-02  46.0  28.0  37.0    12.0      24.0     32.0   
21               3  2012-01-02  45.0  24.0  35.0    -1.0      11.0     28.0   
22               4  2012-01-02  60.0  37.0  49.0    -2.0      26.0     39.0   
23               6  2012-01-02  60.0  30.0  45.0    -4.0      27.0     38.0   
24               7  2012-01-02  44.0  26.0  35.0    -4.0      17.0     29.0   
25               8  2012-01-02  59.0  36.0  35.0    -4.0      29.0     40.0   
26               9  2012-01-02  21.0  11.0  16.0    -4.0       8.0     15.0   
27              10  2012-01-02  65.0  45.0  55.0    -4.0      33.0     46.0   
28              11  2012-01-02  63.0  40.0  52.0    -1.0      31.0     44.0   
29              12  2012-01-02  65.0  47.0  56.0    -1.0      33.0     46.0   
...            ...         ...   ...   ...   ...     ...       ...      ...   
20487           11  2014-10-30  81.0  59.0  70.0     4.0      63.0     65.0   
20488           12  2014-10-30  81.0  61.0  71.0     4.0      63.0     66.0   
20489           13  2014-10-30  61.0  32.0  47.0     4.0      26.0     38.0   
20490           14  2014-10-30  77.0  45.0  61.0     5.0      39.0     50.0   
20491           15  2014-10-30  56.0  30.0  43.0     0.0      36.0     39.0   
20492           16  2014-10-30  57.0  34.0  46.0     0.0      39.0     43.0   
20493           17  2014-10-30  77.0  43.0  60.0     0.0      41.0     50.0   
20494           18  2014-10-30  79.0  56.0  68.0     5.0      51.0     58.0   
20495           19  2014-10-30  51.0  32.0  42.0    -4.0      33.0     40.0   
20496           20  2014-10-30  80.0  52.0  66.0    -4.0      50.0     57.0   
20497            1  2014-10-31  53.0  40.0  47.0    -4.0      36.0     41.0   
20498            2  2014-10-31  49.0  40.0  45.0     0.0      35.0     40.0   
20499            3  2014-10-31  55.0  33.0  44.0   -12.0      32.0     41.0   
20500            4  2014-10-31  76.0  56.0  66.0    -1.0      47.0     56.0   
20501            5  2014-10-31  76.0  56.0  66.0    -1.0      37.0     56.0   
20502            6  2014-10-31  76.0  51.0  64.0     0.0      47.0     55.0   
20503            7  2014-10-31  58.0  38.0  48.0     0.0      32.0     41.0   
20504            8  2014-10-31  78.0  55.0  67.0     0.0      32.0     41.0   
20505            9  2014-10-31  39.0  25.0  32.0     0.0 

In [47]:
def match_store_station():
    key_file = pd.read_csv("Kaggle-Walmart-Sales-Prediction-in-Stormy-Weather-master/Data/key.csv", sep=',')
    df = pd.DataFrame(key_file.values[:,1], index = key_file.values[:,0])
    return df

In [ ]:
def read_processed_weather():
    weather_file = pd.read_csv("Data/weather_processed.csv", sep = ',')
    multi_index = [weather_file.values[:,1], weather_file.values[:,0]]
    df = pd.DataFrame(weather_file.values[:,2:], index=multi_index, columns=weather_file.columns.values[2:])
    return df

In [ ]:
def get_weather_data(date, store_nbr, store_station_pairs, weather_file):
    station_nbr = store_station_pairs.loc[store_nbr].values[0]
    weather_data = weather_file.loc[date, station_nbr].values
    return weather_data

In [ ]:
def get_item_list():
    train_file = pd.read_csv("Data/train.csv", sep=',')
    item_list = list(set(list(train_file.values[:,2])))
    return item_list

In [ ]:
def get_item_data(item_nbr, percent, store_station_pairs, weather_file, train_file):
    start = time.time()
    train_num = train_file.values[:,1].shape[0]

    item_index = np.where(train_file.values[:,2] == item_nbr)
    item_index = item_index[0]
    num = int(item_index.shape[0]*(1-percent)) # Select according to the given percentage, need to be improved
    iter_list = range(num, item_index.shape[0])
    item_dataset = np.zeros((len(iter_list), weather_file.values.shape[1]+1))
    count = 0
    #print len(iter_list)
    for i in iter_list:
        index = item_index[i]
        #print count
        date = train_file.values[:,0][index]
        store_nbr = train_file.values[:,1][index]
        units = train_file.values[:,3][index]

        weather_data = get_weather_data(date, store_nbr, store_station_pairs, weather_file)

        item_dataset[count,:] =  np.append(weather_data, [units], axis=0)
        count += 1
    end = time.time()
    print "Running time for item %d: %f" %(int(item_nbr), (end-start))
    return item_dataset

In [ ]:
def write_item_data(tasks):
    item_list = tasks.get()
    #print item_list
    store_station_pairs = match_store_station()
    weather_file = read_processed_weather() # index has 2 columns and values has 17 columns
    train_file = pd.read_csv("Data/train.csv", sep=',')

    #item_list = get_item_list()
    #item_nbr = item_list[0]
    for item_nbr in item_list:
        item_dataset = get_item_data(item_nbr, 0.1, store_station_pairs, weather_file, train_file) # Give percentage
        df = pd.DataFrame(item_dataset, index=None, columns=None)
        filename = "Item_data/item_%d.csv" %(int(item_nbr))
        df.to_csv(filename, sep=',', index=None, columns=None)

In [ ]:
def multi_process(traget_func):
    processors = multiprocessing.cpu_count()

    myTasks = multiprocessing.Queue()
    item_list = get_item_list()
    temp_part = []
    div = len(item_list)/(processors-1)
    rem = len(item_list)%(processors-1)
    ind = 0
    while ind < div*(processors-1): 
        temp_part.append(item_list[ind:ind+div])
        ind = ind+div
    temp_part.append(item_list[(len(item_list)-rem):])

    for each in temp_part:
        myTasks.put(each)

    Workers = [multiprocessing.Process(target = traget_func, args =(myTasks,)) for i in range(processors)]

    #Workers[0].start()

    for each in Workers:
        each.start()

In [ ]:
def read_item_data(item_nbr):
    filename = "Item_data/item_%d.csv" %(int(item_nbr))
    item_dataset = pd.read_csv(filename, sep=',')

    item_data = item_dataset.values[:,0:-1]
    item_label = item_dataset.values[:,-1]
    return item_data, item_label

In [ ]:
def split_item_data(item_nbr, type):
    item_data, item_label = read_item_data(item_nbr)
    X_train, X_test, y_train, y_test = train_test_split(item_data, item_label, test_size=0.25, random_state=42)
    if type == 'train':
        return X_train, y_train
    elif type == 'test':
        return X_test, y_test

In [ ]:
def get_full_data():
    train_file = pd.read_csv("Data/train.csv", sep=',')

    store_station_pairs = match_store_station()
    weather_file = read_weather() # index has 2 columns and values has 18 columns
    train_num = train_file.values[:,1].shape[0]
    item_dataset = [] # Should be 20 columns
    for ind in range(train_num):
        store_nbr = train_file.values[:,1][ind]
        date = train_file.values[:,0][ind]
        weather_data = get_weather_data(date, store_nbr, store_station_pairs, weather_file)
        weather_data = np.append(train_file.values[:,2:][ind], weather_data, axis = 1) #[item_nbr, units, weather_features]

        item_dataset.append(weather_data)

    item_dataset = np.array(item_dataset)
    train_columns = np.append(np.array(['item_nbr','units']), weather_file.columns.values, axis=1)
    train_df = pd.DataFrame(item_dataset, index=None, columns=train_columns) # Build item_nbr index on units and weather features
    train_df.to_csv("train_feature_matrix.csv", sep=',', index=None)
    return train_df

In [ ]:
def read_full_data():
    train_features = pd.read_csv("train_feature_matrix.csv", sep=',')
    train_df = pd.DataFrame(train_features.values[:,1:], index=train_features.values[:,0], columns=train_features.columns.values[1:])

    print train_df